# 線形回帰 Linear Regression による要因探索とメトリック記録

糖尿病の被験者データに線形回帰を適用し、説明変数・因子がどのように糖尿病の進行に影響しているかを分析します。Scikit Learnを使用し線形回帰で学習します。Azure Machine Learning service のメトリック記録機能を使って、作業内容を記録することも示します。

## Azure Machine Learning service
Azure Machine Learning service の資産管理の仕組みを利用して、メトリック、データセット、モデルを保存します。
### Python SDK インポート

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [ ]:
print("Azure ML SDK Version: ", azureml.core.VERSION)

### ワークスペースへの接続

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

### 実験の開始

In [ ]:
experiment=Experiment(ws, "linear-regression-explore")
# 実験メトリックの記録開始
run = experiment.start_logging()

# 学習データの準備

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

# sklearn の糖尿病のサンプルデータをロード
diabetes = datasets.load_diabetes()
X = pd.DataFrame(diabetes.data, columns=("age", "sex", "bmi", "map", "tc", "ldl", "hdl", "tch", "ltg", "glu"))
y = pd.DataFrame(diabetes.target, columns=["y"])

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100)

# 線形回帰 Linear Regression による学習
[Linear Regression](https://scikit-learn.org/0.20/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) による機械学習モデリング

In [ ]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
lr = lr.fit(X_train, y_train)

### モデル精度の確認

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

pred = lr.predict(X_test)
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test,pred)
print("平均絶対誤差", mae)
print("平均二乗誤差", mse)

## モデルの解釈

In [ ]:
# 回帰係数と切片の大きさ
print(lr.coef_, lr.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lr.coef_[0], ("age", "sex", "bmi", "map", "tc", "ldl", "hdl", "tch", "ltg", "glu"), columns=["Feature Importance"]).abs()
coeff_df.sort_values(by=["Feature Importance"], ascending=False)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.clf()
plt.figure(figsize=(6,6))
coeff_df.plot.bar()
plt.title("feature importance")
plt.legend()
plt.grid(True)
plt.savefig('lr-im.png')
plt.show()

## メトリックの記録

Experiment に紐づけて、利用したハイパーパラメータやモデル精度を記録します。

In [ ]:
# 精度
run.log("平均絶対誤差", mae)
run.log("平均二乗誤差", mse)

# 画像記録
run.log_image("feature importance", "lr-im.png")

### モデル登録

In [ ]:
import pickle
filename = 'lr-model.pkl'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
run.upload_file("outputs/"+filename, filename)

# モデル登録

Azure Machine Learning service 上にモデルを保存します。

In [ ]:
from azureml.core.model import Model

model = run.register_model(model_name='lr-sklearn', 
                           model_path = 'outputs/' + filename,
                           tags = {'area': "sklearn による線形回帰", 'type': "回帰 Regression", 'run_id' : run.id}
                          )

In [ ]:
print(model.name, model.version, model.tags)

## 実験完了

start_logging() で始めた Experiment を終了します。

In [ ]:
run.complete()

In [ ]:
# Git情報などの確認
run.get_details()

In [ ]:
run